In [1]:
import lightgbm as lgb
import pandas as pd
import catboost as cb
import xgboost as xgb

In [2]:
import catboost.datasets

Load data
====

In [4]:
epsilon_test = catboost.datasets.epsilon()[1]

In [6]:
X_test = epsilon_test.drop(0, axis=1).values
xgb_test = xgb.DMatrix(X_test)
catboost_pool = cb.Pool(X_test)

In [8]:
epsilon_model = cb.CatBoost()
epsilon_model.load_model('epsilon8k_64.bin')
print(epsilon_model.tree_count_)

8000


32 thread apply
====

In [9]:
bst = xgb.Booster({'nthread': 32})  # set 32 thread in openmp threadpool
bst.load_model('XGBoost.model')  # load data
lgb_model = lgb.Booster(model_file='LightGBM_model.txt')

In [10]:
%timeit -r 5 bst.predict(xgb_test, ntree_limit=8000)

4.89 s ± 223 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [11]:
%timeit -r 5 lgb_model.predict(X_test, num_iteration=8000)

11.1 s ± 402 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)


In [12]:
%timeit -r 5 epsilon_model.predict(catboost_pool, thread_count=32, ntree_end=8000)

184 ms ± 806 µs per loop (mean ± std. dev. of 5 runs, 1 loop each)


1 thread apply
====

In [13]:
bst = xgb.Booster({'nthread': 1})  # set 1 thread in openmp threadpool
bst.load_model('XGBoost.model')  # load data
lgb_model = lgb.Booster(model_file='LightGBM_model.txt')

In [14]:
%time _ = bst.predict(xgb_test, ntree_limit=8000)

CPU times: user 1min 12s, sys: 20 ms, total: 1min 12s
Wall time: 1min 12s


In [15]:
%time _ = lgb_model.predict(X_test, num_iteration=8000)

CPU times: user 1min 53s, sys: 2.72 s, total: 1min 56s
Wall time: 1min 56s


In [16]:
%time _ = epsilon_model.predict(catboost_pool, thread_count=1, ntree_end=8000)

CPU times: user 1.96 s, sys: 24 ms, total: 1.98 s
Wall time: 1.98 s
